In [3]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

os.environ['HF_ENDPOINT'] = "https://hf-mirror.com"

In [4]:
import string
import numpy as np 
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import re
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input,Flatten,Embedding, Dropout,Conv1D,MaxPooling1D, Dense, GlobalMaxPooling1D,BatchNormalization, Add,GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras import datasets,models,layers
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

In [6]:
data = pd.read_csv('./data/training.1600000.processed.noemoticon.csv',encoding = 'latin',header=None)

In [7]:
df.head()

NameError: name 'df' is not defined

In [8]:
data = data[[5, 0]]
data.columns=['tweet', 'sentiment']

In [9]:
data['sentiment'] = data['sentiment'].replace(4,1)

In [10]:
stop_words = set(stopwords.words('english'))
def preprocess(text):
    # Removing URLS
    text = re.sub(r"https?://\S+|www\.\S+"," ",text)
    
    # Removing html tags
    text = re.sub(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});"," ",text)
    
    # Removing the Punctuation
    text = re.sub(r"[^\w\s]", " ", text)
        
    # Removing words that have numbers 
    text = re.sub(r"\w*\d\w*", " ", text)
        
    # Removing Digits 
    text = re.sub(r"[0-9]+", " ", text)
        
    # Cleaning white spaces
    text = re.sub(r"\s+", " ", text).strip()
        
    text = text.lower()
    # Check stop words
    tokens = []
    for token in text.split():
        if token not in stop_words and len(token) > 3:
            tokens.append(token)
    return " ".join(tokens)

In [11]:
data.loc[:, 'tweet'] = data['tweet'].apply(preprocess)

In [12]:
X = data['tweet']
y = data['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=7)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=2./9, random_state=7)

print("Train Data size:", len(X_train), len(y_train))
print("Validation Data size:", len(X_val), len(y_val))
print("Test Data size", len(X_test), len(y_test))

Train Data size: 1120000 1120000
Validation Data size: 320000 320000
Test Data size 160000 160000


In [13]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

# Convert text to sequences of integers
X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)
X_test = tokenizer.texts_to_sequences(X_test)

In [14]:
max_length = max([len(seq) for seq in X_train])
X_train = pad_sequences(X_train, maxlen=max_length)
X_val = pad_sequences(X_val, maxlen=max_length)
X_test = pad_sequences(X_test, maxlen=max_length)
print(f"After padding: {X_train.shape}")
print(f"After padding: {X_val.shape}")
print(f"After padding:{X_test.shape}")

After padding: (1120000, 27)
After padding: (320000, 27)
After padding:(160000, 27)


In [15]:
# np.save('X_train.npy', X_train)
# np.save('X_val.npy', X_val)
# np.save('X_test.npy', X_test)
# np.save('y_train.npy', y_train)
# np.save('y_val.npy', y_val)
# np.save('y_test.npy', y_test)

In [16]:
# X_train = np.load('./data/X_train.npy')
# y_train = np.load('./data/y_train.npy')
# X_val = np.load('./data/X_val.npy')
# y_val = np.load('./data/y_val.npy')
# X_test = np.load('./data/X_test.npy')
# y_test = np.load('./data/y_test.npy')

# max_length = max([len(seq) for seq in X_train])

# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(X_train)

In [17]:
batch_size=512
# vocab_size = 413238
vocab_size = len(tokenizer.word_index)+1
embedding_dim = 100
vocab_size

413238

In [18]:
CNN=models.Sequential([
    layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    layers.Dropout(0.5),
    
    layers.Conv1D(filters=256, kernel_size=3, activation='relu'), 
    layers.Conv1D(filters=256, kernel_size=3, activation='relu'),
    layers.GlobalMaxPooling1D(),
    
    layers.Flatten(),
    layers.Dense(128,activation="relu"),
    layers.Dropout(0.5),
    
    layers.Dense(1, activation='sigmoid')
])

CNN.compile(loss = 'binary_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])

2024-05-22 00:40:36.321452: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9804 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:3e:00.0, compute capability: 7.5


In [19]:
CNN.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 27, 100)           41323800  
                                                                 
 dropout (Dropout)           (None, 27, 100)           0         
                                                                 
 conv1d (Conv1D)             (None, 25, 256)           77056     
                                                                 
 conv1d_1 (Conv1D)           (None, 23, 256)           196864    
                                                                 
 global_max_pooling1d (Glob  (None, 256)               0         
 alMaxPooling1D)                                                 
                                                                 
 flatten (Flatten)           (None, 256)               0         
                                                        

In [20]:
history=CNN.fit(X_train, y_train, epochs=3,batch_size=batch_size)

Epoch 1/3


2024-05-22 00:40:53.477751: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8700
2024-05-22 00:40:53.730688: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f90d052b5c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-22 00:40:53.730712: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 Ti, Compute Capability 7.5
2024-05-22 00:40:53.735654: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-22 00:40:53.869389: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2188/2188 [==============================] - 67s 29ms/step - loss: 0.5064 - accuracy: 0.7503
Epoch 2/3
2188/2188 [==============================] - 23s 10ms/step - loss: 0.4165 - accuracy: 0.8116
Epoch 3/3
2188/2188 [==============================] - 23s 10ms/step - loss: 0.3479 - accuracy: 0.8461


In [21]:
CNN.evaluate(X_test, y_test)

5000/5000 [==============================] - 6s 1ms/step - loss: 0.6211 - accuracy: 0.7508


[0.6210918426513672, 0.750781238079071]

In [36]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2024-05-21 23:42:17--  http://nlp.stanford.edu/data/glove.6B.zip
Connecting to 100.72.64.19:12798... connected.
Proxy request sent, awaiting response... 503 Service Unavailable
2024-05-21 23:43:38 ERROR 503: Service Unavailable.



In [22]:
from tensorflow.keras.layers import Conv1D, Bidirectional, LSTM, Dense, Input, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.callbacks import ModelCheckpoint

In [34]:
# sequence_input = Input(shape=(max_length,), dtype='int32')
# embedding_sequences = embedding_layer(sequence_input)
# x = SpatialDropout1D(0.2)(embedding_sequences)
# x = Conv1D(64, 5, activation='relu')(x)
# x = Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2))(x)
# x = Dense(512, activation='relu')(x)
# x = Dropout(0.5)(x)
# x = Dense(512, activation='relu')(x)
# outputs = Dense(1, activation='sigmoid')(x)
# model = tf.keras.Model(sequence_input, outputs)

LSTM=models.Sequential([
    layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    layers.SpatialDropout1D(0.2),
    
    layers.Conv1D(64, 5, activation='relu'), 
    layers.Bidirectional(layers.LSTM(64, dropout=0.2, recurrent_dropout=0)),
    
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

LSTM.compile(loss = 'binary_crossentropy',
                optimizer = 'adam',
                metrics = ['accuracy'])

In [35]:
LSTM.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 27, 100)           41323800  
                                                                 
 spatial_dropout1d_5 (Spati  (None, 27, 100)           0         
 alDropout1D)                                                    
                                                                 
 conv1d_7 (Conv1D)           (None, 23, 64)            32064     
                                                                 
 bidirectional_5 (Bidirecti  (None, 128)               66048     
 onal)                                                           
                                                                 
 dense_14 (Dense)            (None, 512)               66048     
                                                                 
 dropout_7 (Dropout)         (None, 512)              

In [36]:
history=LSTM.fit(X_train, y_train, epochs=1,batch_size=batch_size)

2188/2188 [==============================] - 54s 23ms/step - loss: 0.4947 - accuracy: 0.7570


In [26]:
LSTM.evaluate(X_test, y_test)

5000/5000 [==============================] - 51s 10ms/step - loss: 0.4711 - accuracy: 0.7721


[0.4711243212223053, 0.7721375226974487]

In [28]:
def decode_sentiment(score):
    return "Positive" if score>0.5 else "Negative"

scores = LSTM.predict(X_test, verbose=1, batch_size=10000)
y_pred_1d = [decode_sentiment(score) for score in scores]

16/16 [==============================] - 1s 24ms/step


In [29]:
y_pred_1d

['Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Negative',